In [27]:
function route_encrypt(text::String, code::String)
    ind = sortperm(split(code, "")) # массив индексов для перестановки символов
    n = length(code) # строки
    while mod(length(text), n) != 0
        text *= "z"
    end
    println("Обработанный текст: ", text)
    
    m = div(length(text), n) # блоки/строки
    println("Получаем матрицу размерностью nxm: ", n, "x", m)
    x = split(text, "") # массив символов текста
    x_matrix = reshape(x, n, m)
    x_matrix_res = copy(x_matrix)
    for i in 1:n
        x_matrix_res[i, :] = x_matrix[ind[i], :]
    end
    
    return result
end

route_encrypt (generic function with 1 method)

In [28]:
route_encrypt("нельзянедооцениватьпротивника", "пароль")

Обработанный текст: нельзянедооцениватьпротивникаz
Получаем матрицу размерностью nxm: 6x5


"еенпнзоатаьовокннеьвлдирияцтиz"

In [81]:
using Random

In [84]:
function error_h(text)
    flag = false
    while !flag
        try 
            Int(sqrt(length(text))/2)
        catch
            text *= "a"
        else
            flag = true
        end
    end
    return Int(sqrt(length(text))/2), text
end

error_h (generic function with 2 methods)

In [87]:
function grid_encrypt(text::String, code::String, slots::Vector)
    k, text = error_h(text) # вызываем обработку текста, получаем k и обработанный текст
    # print(k)
    if k<2 # проверка
        return println("k<2 шифрование невозможно")
    end
    x = split(text, "") # массив символов текста
    
    # после обработки текста обрабатываем код:
    if length(code) > 2*k # если код длинный
        code = code[1:2*k] # обрезаем
    elseif length(code) < 2*k  # если код короткий
        while length(code) < 2*k # дополняем до нужной длины
            code *= "a"
        end
    end

    # обработка прорезей
    if length(slots) != k^2 # если размер прорезей не соответствует
        slots = rand(1:4, k^2) # генерим случайные значения
    end

    slots_q = Array{Integer, 2}(undef, 2*k, 2*k) #инициализировали массив прорезей
    # 4 сектора маски
    slots_q[1:k, 1:k] = [slots[i+k*j] == 1 ? i+k*j : 0 for i=1:k, j=0:k-1]
    slots_q[1:k, k+1:2*k] = [slots[i+k*j] == 2 ? i+k*j : 0 for j=0:k-1,i=k:-1:1]
    slots_q[k+1:2*k, k+1:2*k] = [slots[i+k*j] == 3 ? i+k*j : 0 for j=k-1:-1:0, i=k:-1:1]
    slots_q[k+1:2*k, 1:k] = [slots[i+k*j] == 4 ? i+k*j : 0 for j=k-1:-1:0,i=1:k]
    text_m = Array{String, 2}(undef, 2*k, 2*k) 
    for i in 1:4 # заполняем массив текстом
        for j in 1:k^2 # заполнили прорези
            text_m[findfirst( x -> x== j, slots_q)] = x[(i-1)*k^2+j]
        end
        slots_q = rotr90(slots_q) # повернули на 90
    end
    
    ind = sortperm(split(code, "")) # сортируем по коду
    # перестанавливаем столбцы
    res = copy(text_m)
    for i in 1:2*k
        res[i, :] = text_m[ind[i], :]
    end
    result=""
    for i in 1:2*k
        result *= join(res[i,:])
    end
    return result, code, slots
end

grid_encrypt (generic function with 1 method)

In [88]:
encrypted_text, code, slots = grid_encrypt("договорподписали", "шифр", [2, 3, 3, 4])

("ппиаоровооигсдлд", "шифр", [2, 3, 3, 4])

In [123]:
function vigenere_encrypt(text::String, key::String, mode::Symbol=:encrypt)
    #создаем алфавит
    alphabet = 'A':'Z'
    text = uppercase(text) # приводим к верхнему регистру текст
    key = uppercase(key) # и ключ
    # все инородные сиволы удаляем (не входящие в алфавит)
    text = filter(c -> c in alphabet, text)
    result = []
    key_length = length(key)
    key_ind = 1

    # сдвиг либо 1 либо -1 для шифрования/дешифрования
    shift = mode == :encrypt ? 1 : -1

    for char in text
        # позиции символа и ключа
        char_pos = findfirst(isequal(char), alphabet)
        key_char = key[key_ind]
        key_pos = findfirst(isequal(key_char),alphabet)
        # новые позиции
        new_pos = mod1(char_pos + shift * key_pos, length(alphabet))
        # довавляем
        push!(result, alphabet[new_pos])
        key_ind = key_ind % key_length + 1
    end
    
    return join(result)
end

vigenere_encrypt (generic function with 3 methods)

In [124]:
vigenere_encrypt("beautiful sunny day", "master")

"OFTOYASVEMZFAZWUD"